In [3]:
from langchain_community.document_loaders import HuggingFaceDatasetLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from transformers import AutoTokenizer, AutoModelForCausalLM,pipeline
from langchain_huggingface import HuggingFacePipeline
# RetrievalQA is deprecated in newer LangChain versions
# Use create_retrieval_chain or LCEL pattern instead
# from langchain.chains import RetrievalQA  # Deprecated

/Users/taha/Projects/langchain-course/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
from dotenv import load_dotenv
import os
from huggingface_hub import login

# Load environment variables from .env file
load_dotenv()

# Get Hugging Face token from .env file
hf_token = os.getenv("HF_TOKEN") or os.getenv("HUGGINGFACE_TOKEN") or os.getenv("HUGGINGFACE_HUB_TOKEN")

if not hf_token:
    raise ValueError("HF_TOKEN, HUGGINGFACE_TOKEN, or HUGGINGFACE_HUB_TOKEN not found in .env file")

# Login to Hugging Face
login(token=hf_token)

Note: Environment variable`HF_TOKEN` is set and is the current active token independently from the token you've just configured.


In [5]:
# Specify the dataset name and the column containing the content
dataset_name = "ruslanmv/ai-medical-chatbot"


# Create a loader instance
loader_doctor = HuggingFaceDatasetLoader(dataset_name,"Doctor")

In [6]:
# # Load the data
doctor_data = loader_doctor.load()

# Select the first 1000 entries
doctor_data = doctor_data[:1000]

doctor_data[:2]

Generating train split: 100%|██████████| 256916/256916 [00:00<00:00, 1591777.18 examples/s]


[Document(metadata={'Description': 'Q. What does abutment of the nerve root mean?', 'Patient': 'Hi doctor,I am just wondering what is abutting and abutment of the nerve root means in a back issue. Please explain. What treatment is required for\xa0annular bulging and tear?'}, page_content='"Hi. I have gone through your query with diligence and would like you to know that I am here to help you. For further information consult a neurologist online -->"'),
 Document(metadata={'Description': 'Q. What should I do to reduce my weight gained due to genetic hypothyroidism?', 'Patient': 'Hi doctor, I am a 22-year-old female who was diagnosed with hypothyroidism (genetic) when I was 12. Over the past five years, I have become around 50 pounds overweight and all of my attempts to lose have seemed to fail so I have given up, but my weight has stayed the same. There is so much information put there about losing weight with hypothyroidism but it all seems to conflict. I am so unsure as to what type o

In [9]:
# Define the path to the pre-trained model you want to use
modelPath = "sentence-transformers/all-MiniLM-L12-v2"

# Create a dictionary with model configuration options, specifying to use the GPU for computations
model_kwargs = {'device':'cpu'}

# Create a dictionary with encoding options, specifically setting 'normalize_embeddings' to False
encode_kwargs = {'normalize_embeddings': False}

# Initialize an instance of HuggingFaceEmbeddings with the specified parameters
embeddings = HuggingFaceEmbeddings(
    model_name=modelPath,     
    model_kwargs=model_kwargs, 
    encode_kwargs=encode_kwargs
)

In [10]:
text = "Why are you a doctor?"
query_result = embeddings.embed_query(text)
query_result[:3]

[-0.05935199186205864, 0.08008933812379837, 0.040729619562625885]

In [13]:
vector_db = FAISS.from_documents(doctor_data, embeddings)
vector_db.save_local("db/faiss_doctor_index")

In [14]:
question = "Hi Doctor, I have a headache, help me."
searchDocs = vector_db.similarity_search(question)
print(searchDocs[0].page_content)

"Hello. You have a persistent headache in the frontal part of your head. Was there fever, chills, and rigor, were you feeling like vomiting, after taking those drugs prescribed for you? Did your headache subside? The medicines given at the pharmacy are fine for malaria, but it may not have been malaria, but it could be Salmonella typhi infection, which causes typhoid fever. Worm medicine is fine in the case. I will advise you to go to a lab and do blood film for malaria parasite, do Widal test and stool microscopy culture and sensitivity. If there is salmonella infection E coli or any enterobacteria infection, the stool MCS will detect it. If there is worm infestation such as hookworm or tapeworm, it will also pick it in microscopy. You can take tablet Buscopan (hyoscine butylbromide) 10 mg thrice daily. Then get tablet Ciprofloxacin 500 mg bd (twice daily) take them for five days. You can add tablet Rabeprazole 20 mg bd in case if there is any peptic ulcer since the description of the

In [15]:
import torch
base_model = "/kaggle/input/fine-tuned-adapter-to-full-model/llama-3-8b-chat-doctor/"

tokenizer = AutoTokenizer.from_pretrained(base_model)

model = AutoModelForCausalLM.from_pretrained(
        base_model,
        return_dict=True,
        low_cpu_mem_usage=True,
        torch_dtype=torch.float16,
        device_map="auto",
        trust_remote_code=True,
)

HFValidationError: Repo id must be in the form 'repo_name' or 'namespace/repo_name': '/kaggle/input/fine-tuned-adapter-to-full-model/llama-3-8b-chat-doctor/'. Use `repo_type` argument if needed.

In [16]:
pipe = pipeline(
    "text-generation", 
    model=model, 
    tokenizer=tokenizer,
    max_new_tokens=120
)

llm = HuggingFacePipeline(pipeline=pipe)

NameError: name 'model' is not defined

In [ ]:
retriever = vector_db.as_retriever()
